In [112]:
##This notebook is my attempt to use a different Folium class .GeoJson vs .Choropleth to build the choropleth

import os
import folium
from folium import Map, Marker, GeoJson, LayerControl, FeatureGroup
import urllib
import requests
import json
import geopandas as gp
from pathlib import Path
import pandas as pd
import fiona
import branca

In [114]:
##Importing GeoJson file with FIPS.  Later to be join to other dataFrame to create GeoDataFrame
harris_co_geo = gp.read_file(
    '../data/cleandata/harris_co_clean.geojson',
    driver='GeoJSON',
    dtype={"FIPS": str}
)


# with open('../data/cleandata/harris_co_clean.geojson') as response:
#     geo_data = json.load(response)

In [116]:
##ALL DATA
join_df = pd.read_csv('../data/cleandata/join_data.csv',dtype={"FIPS": str})
join_df.head()

##SVI DATA TO DataFrame and CLEANED
svi_df = join_df[['FIPS','SVI INDEX']]
svi_df = svi_df.rename(columns={'FIPS':'FIPS','SVI INDEX':'SVI'})
svi_df = svi_df[(svi_df['SVI'] >= 0) & (svi_df['SVI'] <= 1)]

##school DATA TO CHOROPLETH IMPORTED AND CLEANED
school_df = join_df[['FIPS','Rating']]

# FLOOD DATA
flood_df = join_df[['FIPS','Pct_in_100yr']]

## EVICTION DATA
eviction_df = join_df[['FIPS','eviction-rate']]

## FOOD DESERT
food_desert_df = join_df[['FIPS','food_desert']]

# POVERTY RATE
poverty_df = join_df[['FIPS','poverty-rate']]


## MEDIAN PROPERTY VALUE
property_value_df = join_df[['FIPS','median-property-value']]

## ROAD CLOSURES
road_df = pd.read_csv('../data/cleandata/road_flooding_imelda.csv',dtype={"FIPS": str})

In [118]:
geo_df = harris_co_geo.merge(svi_df, on='FIPS')
geo_df.to_file('../data/rawdata/geo_test.json', driver="GeoJSON")

In [125]:
with open('../data/rawdata/geo_test.json') as response:
    geo_data = json.load(response)
geo_data['features'][1]

{'type': 'Feature',
 'properties': {'FIPS': '48201210900', 'SVI': 0.7334},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-95.324227, 29.809072],
     [-95.324093, 29.800165],
     [-95.329557, 29.800098],
     [-95.331362, 29.800075],
     [-95.334864, 29.800172],
     [-95.340085, 29.801501],
     [-95.339519, 29.804216],
     [-95.336254, 29.808262],
     [-95.333024, 29.807964],
     [-95.32851, 29.808462],
     [-95.324227, 29.809072]]]]}}

In [176]:
from branca.colormap import linear

flood_colors = linear.YlGn_05.scale(
    flood_df['Pct_in_100yr'].min(),
    flood_df['Pct_in_100yr'].max(), 
)


flood_colors

In [177]:
map = folium.Map([29.7604, -95.358421], tiles = "OpenStreetMap", zoom_start=10)
flood_colors.caption = 'Percentage of Home in 100yr Flood Plain'
flood_colors.add_to(map)

feature_group_SVI = folium.FeatureGroup(name="SVI",opacity = 1)

GeoJson(geo_data,
        name = 'SVI',
        style_function = lambda features: {
            'color' : 'grey',
            'weight': .5,
            'fillColor' : flood_colors(features['properties']['SVI']),
            'fillOpacity' : .7
            }
        ).add_to(feature_group_SVI)
feature_group_SVI.add_to(map)
folium.LayerControl().add_to(map)




map

In [110]:
# map = folium.Map([29.7604, -95.358421], tiles = "OpenStreetMap", zoom_start=10)

# def style_function(feature):
#     eviction = employed_series.get(feature)
#     return {
#         'fillOpacity': 0.5,
#         'weight': 0,
#         'fillColor': '#black' if eviction is None else colorscale(employed)
#     }
# folium.GeoJson(
#     tracts_data,
#     'feature.properties.FIPS',
#     style_function=style_function
# ).add_to(map)
# map